In [1]:
%load_ext autoreload
%autoreload 2

In [32]:
import models
from sklearn.metrics.pairwise import haversine_distances
import numpy as np
import plotly
import plotly.express as px
import pandas as pd
import geopandas as gpd
import mscthesis

In [3]:
geotweets = mscthesis.read_geotweets('sweden').set_index('userid')

In [14]:
tweetcount = geotweets.groupby('userid').size()
tweetcount

userid
5616                  2130
5976                   287
8981                   941
32213                  413
392123                 617
                      ... 
764655612525473792      64
778179816889675777      36
782260642770288640      47
790301765744951296      54
794973407964229636     856
Length: 4056, dtype: int64

In [24]:
geotweets = geotweets.drop(
    labels=tweetcount[tweetcount < 50].index,
)

In [27]:
samples = []
def save_sample(uid, day, timeslot, d, sample):
    s = [uid, day, timeslot]
    if sample[0] == 'region':
        s = s + ['region', sample[1], sample[2], sample[3]]
    else:
        s = s + ['point', -1, sample[1], sample[2]]
    d.append(s)

i = 0
for uid in geotweets.index.unique():
    usertweets = geotweets.loc[uid]
    m = models.SongModel(p=0.66, gamma=0.6, zipf=-1.2)
    m.fit(usertweets)
    home_location = usertweets[usertweets['label'] == 'home']
    for day in range(7*20):
        prev = (
            "region", 
            home_location.iloc[0]['region'], 
            home_location.iloc[0]['latitude'], 
            home_location.iloc[0]['longitude'],
        )
        save_sample(uid, day, 0, samples, prev)
        for timeslot in range(3):
            prev = m.sample(prev)
            save_sample(uid, day, timeslot+1, samples, prev)
    i += 1
    if i % 250 == 0:
        print("done with", i)

done with 250
done with 500
done with 750
done with 1000
done with 1250
done with 1500
done with 1750
done with 2000
done with 2250
done with 2500
done with 2750
done with 3000
done with 3250


In [28]:
samplesx = pd.DataFrame(
    samples,
    columns=['userid', 'day', 'timeslot', 'kind', 'region', 'latitude', 'longitude'],
).set_index('userid')

In [33]:
samplesx.loc[5616]

,day,timeslot,kind,region,latitude,longitude
userid,,,,,,
5616,0,0,region,5,59.426889,17.954336
5616,0,1,region,5,59.426889,17.954336
5616,0,2,region,347,59.445959,17.942841
5616,0,3,region,5,59.426889,17.954336
5616,1,0,region,5,59.426889,17.954336
...,...,...,...,...,...,...
5616,138,3,region,4,59.452536,17.930460
5616,139,0,region,5,59.426889,17.954336
5616,139,1,region,144,49.004012,2.571058


In [34]:
samplesx.to_csv('./../../dbs/sweden/visits-song.csv')